In [1]:
import socket
import mediapipe as mp
import cv2
import pickle
import threading
import asyncio
import pandas as pd
from dollarpy import Point
from bleak import BleakScanner
from openpyxl import load_workbook

HOSTNAME = '127.0.0.1'
PORT = 5010

# Path to the Excel file
excel_path = r"C:\Users\Lenovo\Videos\Smart_Shopping-main\bluetooth_devices.xlsx"

# Load registered MAC addresses from Excel
try:
    mac_df = pd.read_excel(excel_path)
    registered_macs = set(mac_df['MAC Address'].dropna().astype(str).tolist())
except Exception as e:
    print("Error reading Excel file:", e)
    registered_macs = set()  # Start with an empty set if there’s an issue

# Socket setup
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.bind((HOSTNAME, PORT))
soc.listen(5)
conn, addr = soc.accept()
print("Device connected")

async def scan_for_devices():
    devices = await BleakScanner.discover()
    new_entries = []

    for device in devices:
        mac_address = device.address
        device_name = device.name
        print(f"Device: {device_name}, MAC Address: {mac_address}")
        
        # Check if MAC address is already registered
        if mac_address in registered_macs:
            msg_pred = f"logged in\n".encode("utf-8")
        else:
            # Register new MAC address and update the registered set
            msg_pred = f"registered\n".encode("utf-8")
            registered_macs.add(mac_address)
            new_entries.append({'MAC Address': mac_address, 'Device Name': device_name})  # Add to new entries
        
        # Send message
        conn.send(msg_pred)

    # Append new entries directly to the Excel file if there are any
    if new_entries:
        try:
            workbook = load_workbook(excel_path)
            sheet = workbook.active
            
            # Find the next empty row and append new entries
            for entry in new_entries:
                sheet.append([entry['MAC Address'], entry['Device Name']])
            
            # Save the workbook
            workbook.save(excel_path)
            print("Updated Excel with new registered devices.")
        
        except Exception as e:
            print("Error updating Excel file:", e)

def scan_in_background():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(scan_for_devices())

scanner_thread = threading.Thread(target=scan_in_background)

scanner_thread.start()

scanner_thread.join()

conn.send(bytes("q", "utf-8"))
conn.close()
soc.close()


Device connected
Device: net, MAC Address: D8:34:1C:FA:05:97
Device: None, MAC Address: 31:2A:01:04:46:9E
Updated Excel with new registered devices.


In [1]:
!pip install deepface

In [1]:
import socket
import cv2
import pickle
import threading
from queue import Queue
from dollarpy import Point
import mediapipe as mp
from ultralytics import YOLO
import time
from deepface import DeepFace
from simple_facerec import SimpleFacerec
import pandas as pd

HOSTNAME = '127.0.0.1'
PORT = 5010

# Path to the Excel file
excel_path = r"C:\HCI\Smart_Shopping-main (8)\Smart_Shopping-main\bluetooth_devices.xlsx"

# Load registered MAC addresses from Excel
try:
    mac_df = pd.read_excel(excel_path)
    registered_macs = set(mac_df['MAC Address'].dropna().astype(str).tolist())
except Exception as e:
    print("Error reading Excel file:", e)
    registered_macs = set()  # Start with an empty set if there’s an issue

# Socket setup
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.bind((HOSTNAME, PORT))
soc.listen(5)
conn, addr = soc.accept()
print("Device connected")


# Initialize the camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video device.")
    exit(1)
else:
    print("Camera successfully opened.")

# Initialize YOLO model
model = YOLO(r"C:\HCI\Smart_Shopping-main (8)\Smart_Shopping-main\best (2).pt")
target_class_names = ["deodorant", "vitamin"]
target_classes = [index for index, name in model.names.items() if name in target_class_names]

# Initialize MediaPipe Hands
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize face recognition
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")

# Lock for thread synchronization
camera_lock = threading.Lock()
socket_lock = threading.Lock()

# Frame queue for threads
frame_queue = Queue(maxsize=10)

# Shutdown flag
shutdown_flag = False

def send_data(msg):
    try:
        with socket_lock:
            if conn.fileno() != -1:
                conn.send(msg)
            else:
                print("Socket is closed.")
    except socket.error as e:
        print(f"Socket error: {e}")
        global shutdown_flag
        shutdown_flag = True
        conn.close()

def capture_frame():
    global shutdown_flag
    while not shutdown_flag:
        with camera_lock:
            ret, frame = cap.read()
            if ret:
                if frame_queue.full():
                    frame_queue.get()
                frame_queue.put(frame)
            else:
                print("Failed to capture frame.")
                shutdown_flag = True
                break

def capture_gestures():
    global shutdown_flag  # Ensure we're modifying the global shutdown_flag
    with open('gesture_recognizer1.pkl', 'rb') as file:
        recognizer = pickle.load(file)
    
    points = []
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()

            # Convert the frame to RGB for MediaPipe processing
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            annotated_image = frame.copy()

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    
                    # Capture important hand landmarks for gesture recognition
                    wrist = Point(hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y, 1)
                    thumb_cmc = Point(hand_landmarks.landmark[1].x, hand_landmarks.landmark[1].y, 1)
                    thumb_mcp = Point(hand_landmarks.landmark[2].x, hand_landmarks.landmark[2].y, 1)
                    thumb_ip = Point(hand_landmarks.landmark[3].x, hand_landmarks.landmark[3].y, 1)
                    thumb_tip = Point(hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y, 1)

                    points = [wrist, thumb_cmc, thumb_mcp, thumb_ip, thumb_tip]

                    print(f"Hand landmarks: {points}")  # Debug: Log the landmarks
                    try:
                        # Check if the recognizer is working and making a prediction
                        prediction = recognizer.recognize(points)
                        print(f"Predictions: {prediction}")  # Debugging
                        
                        if prediction and len(prediction) > 0:
                            detected_gesture = prediction[0]
                            print(f"Detected gesture: {detected_gesture}")  # Debugging
                            msg_pred = f"{detected_gesture}".encode("utf-8")
                            if conn.fileno() != -1:  # Check if the connection is still open
                                send_data(msg_pred)  # Send data to the client
                            else:
                                print("Connection closed, unable to send gesture data.")
                    except Exception as e:
                        print("An error occurred:", str(e))
            else:
                print("No hand landmarks detected.")

            # Display the annotated image with landmarks
            cv2.imshow('Camera - Gesture Recognition', annotated_image)

            # Exit if the 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break


def detect_face_and_emotions():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            flipped_frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(flipped_frame, cv2.COLOR_BGR2RGB)
            face_locations, face_names = sfr.detect_known_faces(flipped_frame)

            for face_loc, name in zip(face_locations, face_names):
                y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
                face_roi = rgb_frame[y1:y2, x1:x2]
                try:
                    emotion_result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
                    emotion = emotion_result[0]['dominant_emotion']
                except Exception as e:
                    emotion = "Unknown"
                display_text = f"{name} is {emotion}"
                cv2.putText(flipped_frame, display_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 200), 2)
                cv2.rectangle(flipped_frame, (x1, y1), (x2, y2), (0, 0, 200), 2)

            cv2.imshow("Face Recognition and Emotion Detection", flipped_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

def detect_objects():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            frame_resized = cv2.resize(frame, (640, 640))
            results = model(frame_resized, conf=0.9, classes=target_classes)

            annotated_frame = results[0].plot()
            if len(results[0].boxes.cls) > 0:
                for box, score, cls in zip(results[0].boxes.xywh, results[0].boxes.conf, results[0].boxes.cls):
                    class_name = model.names[int(cls)]
                    confidence = float(score)
                    if class_name in target_class_names:
                        msg_class = f"Detected Object: {class_name}".encode("utf-8")
                        send_data(msg_class)

            cv2.imshow("Object Detection", annotated_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

async def scan_for_devices():
    devices = await BleakScanner.discover()
    new_entries = []

    for device in devices:
        mac_address = device.address
        device_name = device.name
        print(f"Device: {device_name}, MAC Address: {mac_address}")
        
        # Check if MAC address is already registered
        if mac_address in registered_macs:
            msg_pred = f"logged in\n".encode("utf-8")
        else:
            # Register new MAC address and update the registered set
            msg_pred = f"registered\n".encode("utf-8")
            registered_macs.add(mac_address)
            new_entries.append({'MAC Address': mac_address, 'Device Name': device_name})  # Add to new entries
        
        # Send message
        conn.send(msg_pred)

    # Append new entries directly to the Excel file if there are any
    if new_entries:
        try:
            workbook = load_workbook(excel_path)
            sheet = workbook.active
            
            # Find the next empty row and append new entries
            for entry in new_entries:
                sheet.append([entry['MAC Address'], entry['Device Name']])
            
            # Save the workbook
            workbook.save(excel_path)
            print("Updated Excel with new registered devices.")
        
        except Exception as e:
            print("Error updating Excel file:", e)

def scan_in_background():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(scan_for_devices())

# Start threads
camera_thread = threading.Thread(target=capture_frame)
face_emotion_thread = threading.Thread(target=detect_face_and_emotions)
gesture_thread = threading.Thread(target=capture_gestures)
object_detection_thread = threading.Thread(target=detect_objects)
scanner_thread = threading.Thread(target=scan_in_background)

camera_thread.start()
face_emotion_thread.start()
gesture_thread.start()
object_detection_thread.start()
scanner_thread.start()

camera_thread.join()
face_emotion_thread.join()
gesture_thread.join()
object_detection_thread.join()
scanner_thread.join()

cap.release()
cv2.destroyAllWindows()
conn.send(bytes("q", "utf-8"))
conn.close()
soc.close()


Device connected
Camera successfully opened.
10 encoding images found.


Exception in thread Thread-12 (scan_in_background):
Traceback (most recent call last):
  File "c:\Users\alie0\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\alie0\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\alie0\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\alie0\AppData\Local\Temp\ipykernel_32660\3691811562.py", line 238, in scan_in_background
NameError: name 'asyncio' is not defined


Encoding images loaded
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detecte

Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: C:\Users\alie0\.deepface\weights\facial_expression_model_weights.h5


No hand landmarks detected.
Hand landmarks: [(0.29748672246932983, 0.8637678027153015), stroke 1, (0.34389418363571167, 0.8356517553329468), stroke 1, (0.39589935541152954, 0.808243453502655), stroke 1, (0.4440389573574066, 0.8011246919631958), stroke 1, (0.48240941762924194, 0.7997262477874756), stroke 1]
Predictions: (None, 0)
Detected gesture: None


...

 79%|███████▉  | 4.72M/5.98M [00:01<00:00, 3.87MB/s]

No hand landmarks detected.
No hand landmarks detected.


100%|██████████| 5.98M/5.98M [00:01<00:00, 3.43MB/s]

No hand landmarks detected.


No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
Hand landmarks: [(0.4646475613117218, 0.7546353936195374), stroke 1, (0.5035494565963745, 0.7804774045944214), stroke 1, (0.5426828265190125, 0.8099303245544434), stroke 1, (0.5685304999351501, 0.8353204131126404), stroke 1, (0.5870230197906494, 0.8601717948913574), stroke 1]
Predictions: ('Peace', 0.050336520515353866)
Detected gesture: Peace
No hand landmarks detected.
0: 416x416 (no detections), 2308.3ms
Speed: 7.0ms preprocess, 2308.3ms inference, 3.2ms postprocess per image at shape (1, 3, 416, 416)
No hand landmarks detected.

No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landm

In [3]:
!pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 2.1 MB/s eta 0:00:01
   ------------------------ --------------- 1.0/1.7 MB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 2.3 MB/s eta 0:00:00


In [4]:
!pip install --upgrade deepface


In [1]:
import socket
import cv2
import pickle
import threading
from queue import Queue
from dollarpy import Point
import mediapipe as mp
from ultralytics import YOLO
import time
from deepface import DeepFace
from simple_facerec import SimpleFacerec
import pandas as pd
import bluetooth
import asyncio
from openpyxl import load_workbook  # For Excel file manipulation

HOSTNAME = '127.0.0.1'
PORT = 5010

# Path to the Excel file
excel_path = r"C:\HCI\Smart_Shopping-main (8)\Smart_Shopping-main\bloutooth.xlsx"

# Load registered MAC addresses from Excel
try:
    mac_df = pd.read_excel(excel_path)
    registered_macs = set(mac_df['mac_address'].dropna().astype(str).tolist())
except Exception as e:
    print("Error reading Excel file:", e)
    registered_macs = set()  # Start with an empty set if there’s an issue

# Socket setup
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.bind((HOSTNAME, PORT))
soc.listen(5)
conn, addr = soc.accept()
print("Device connected")

# Initialize the camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video device.")
    exit(1)
else:
    print("Camera successfully opened.")

# Initialize YOLO model
model = YOLO(r"C:\HCI\Smart_Shopping-main (8)\Smart_Shopping-main\best (2).pt")
target_class_names = ["deodorant", "vitamin"]
target_classes = [index for index, name in model.names.items() if name in target_class_names]

# Initialize MediaPipe Hands
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize face recognition
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")

# Lock for thread synchronization
camera_lock = threading.Lock()
socket_lock = threading.Lock()

# Frame queue for threads
frame_queue = Queue(maxsize=10)

# Shutdown flag
shutdown_flag = False

def send_data(msg):
    try:
        with socket_lock:
            if conn.fileno() != -1:
                conn.send(msg)
            else:
                print("Socket is closed.")
    except socket.error as e:
        print(f"Socket error: {e}")
        global shutdown_flag
        shutdown_flag = True
        conn.close()

def capture_frame():
    global shutdown_flag
    while not shutdown_flag:
        with camera_lock:
            ret, frame = cap.read()
            if ret:
                if frame_queue.full():
                    frame_queue.get()
                frame_queue.put(frame)
            else:
                print("Failed to capture frame.")
                shutdown_flag = True
                break

def capture_gestures():
    global shutdown_flag  # Ensure we're modifying the global shutdown_flag
    with open('gesture_recognizer1.pkl', 'rb') as file:
        recognizer = pickle.load(file)
    
    points = []
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()

            # Convert the frame to RGB for MediaPipe processing
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            annotated_image = frame.copy()

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    
                    # Capture important hand landmarks for gesture recognition
                    wrist = Point(hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y, 1)
                    thumb_cmc = Point(hand_landmarks.landmark[1].x, hand_landmarks.landmark[1].y, 1)
                    thumb_mcp = Point(hand_landmarks.landmark[2].x, hand_landmarks.landmark[2].y, 1)
                    thumb_ip = Point(hand_landmarks.landmark[3].x, hand_landmarks.landmark[3].y, 1)
                    thumb_tip = Point(hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y, 1)

                    points = [wrist, thumb_cmc, thumb_mcp, thumb_ip, thumb_tip]

                    print(f"Hand landmarks: {points}")  # Debug: Log the landmarks
                    try:
                        # Check if the recognizer is working and making a prediction
                        prediction = recognizer.recognize(points)
                        print(f"Predictions: {prediction}")  # Debugging
                        
                        if prediction and len(prediction) > 0:
                            detected_gesture = prediction[0]
                            print(f"Detected gesture: {detected_gesture}")  # Debugging
                            msg_pred = f"{detected_gesture}".encode("utf-8")
                            if conn.fileno() != -1:  # Check if the connection is still open
                                send_data(msg_pred)  # Send data to the client
                            else:
                                print("Connection closed, unable to send gesture data.")
                    except Exception as e:
                        print("An error occurred:", str(e))
            else:
                print("No hand landmarks detected.")

            # Display the annotated image with landmarks
            cv2.imshow('Camera - Gesture Recognition', annotated_image)

            # Exit if the 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

def detect_face_and_emotions():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            flipped_frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(flipped_frame, cv2.COLOR_BGR2RGB)
            face_locations, face_names = sfr.detect_known_faces(flipped_frame)

            for face_loc, name in zip(face_locations, face_names):
                y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
                face_roi = rgb_frame[y1:y2, x1:x2]
                try:
                    emotion_result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
                    emotion = emotion_result[0]['dominant_emotion']
                except Exception as e:
                    emotion = "Unknown"
                display_text = f"{name} is {emotion}"
                cv2.putText(flipped_frame, display_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 200), 2)
                cv2.rectangle(flipped_frame, (x1, y1), (x2, y2), (0, 0, 200), 2)

            cv2.imshow("Face Recognition and Emotion Detection", flipped_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

def detect_objects():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            frame_resized = cv2.resize(frame, (640, 640))
            results = model(frame_resized, conf=0.9, classes=target_classes)

            annotated_frame = results[0].plot()
            if len(results[0].boxes.cls) > 0:
                for box, score, cls in zip(results[0].boxes.xywh, results[0].boxes.conf, results[0].boxes.cls):
                    class_name = model.names[int(cls)]
                    confidence = float(score)
                    if class_name in target_class_names:
                        msg_class = f"Detected Object: {class_name}".encode("utf-8")
                        send_data(msg_class)

            cv2.imshow("Object Detection", annotated_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

# Bluetooth function using asyncio
async def continuous_bluetooth_login(scan_duration=5, cycles=5):
    print(f"Starting continuous scanning for Bluetooth devices (each scan lasts {scan_duration} seconds)...")

    try:
        while True:  # Continuous scanning loop
            print("Scanning for Bluetooth devices...")
            all_devices = []  # List to collect all devices detected in the current cycle

            # Perform multiple scan cycles per round
            for _ in range(cycles):
                # Start a Bluetooth scan for Classic Bluetooth devices (both BLE and Classic)
                nearby_devices = bluetooth.discover_devices(duration=scan_duration, lookup_names=True, device_id=-1, flush_cache=True)

                for addr, name in nearby_devices:
                    all_devices.append((addr, name))

            # Remove duplicates based on MAC address
            unique_devices = {device[0]: device[1] for device in all_devices}.items()

            if not unique_devices:
                print("No devices found during this scan.")
            else:
                # Loop through detected devices
                for mac_address, device_name in unique_devices:
                    print(f"Detected device: {device_name}, MAC Address: {mac_address}")

                    # Check if the MAC address matches a registered user
                    if mac_address in registered_macs:
                        # Fetch user details from the Excel file
                        user_data = mac_df.loc[mac_df["MAC Address"] == mac_address]
                        user = user_data.iloc[0]  # Get the matched user profile
                        
                        print(f"Login successful for user: {user['Name']}")
                        print(f"Role: {user['Role']}")
                    else:
                        print(f"Unregistered device detected: {mac_address} (Name: {device_name}).\n")

            print("Scan complete. Restarting scan in 5 seconds...\n")
            await asyncio.sleep(5)  # Pause before the next scan cycle
    except Exception as e:
        print(f"An error occurred during Bluetooth scanning: {e}")

def scan_in_background():
    try:
        asyncio.run(continuous_bluetooth_login(scan_duration=5))
    except RuntimeError as e:
        if "This event loop is already running" in str(e):
            print("Event loop is already running. Using ensure_future().")
            asyncio.ensure_future(continuous_bluetooth_login(scan_duration=5))

# Start threads
camera_thread = threading.Thread(target=capture_frame)
face_emotion_thread = threading.Thread(target=detect_face_and_emotions)
gesture_thread = threading.Thread(target=capture_gestures)
object_detection_thread = threading.Thread(target=detect_objects)
scanner_thread = threading.Thread(target=scan_in_background)
camera_thread.start()
face_emotion_thread.start()
gesture_thread.start()
object_detection_thread.start()
scanner_thread.start()
camera_thread.join()
face_emotion_thread.join()
gesture_thread.join()
object_detection_thread.join()
scanner_thread.join()
cap.release()
cv2.destroyAllWindows()
conn.send(bytes("q", "utf-8"))
conn.close()
soc.close()



Device connected
Camera successfully opened.
10 encoding images found.
Encoding images loaded
Starting continuous scanning for Bluetooth devices (each scan lasts 5 seconds)...
Scanning for Bluetooth devices...
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.

No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No hand landmarks detected.
No h